<p align="center">
  <img src="./imgs/01.png" alt="imagen01" />
</p>

<p align="center">
  <img src="./imgs/02.png" alt="imagen02" />
</p>

<p align="center">
  <img src="./imgs/03.png" alt="imagen03" />
</p>

<p align="center">
  <img src="./imgs/04.png" alt="imagen04" />
</p>

<p align="center">
  <img src="./imgs/05.png" alt="imagen05" />
</p>

<p align="center">
  <img src="./imgs/06.png" alt="imagen06" />
</p>

<p align="center">
  <img src="./imgs/07.png" alt="imagen07" />
</p>

<p align="center">
  <img src="./imgs/08.png" alt="imagen08" />
</p>

<p align="center">
  <img src="./imgs/09.png" alt="imagen09" />
</p>

<p align="center">
  <img src="./imgs/10.png" alt="imagen10" />
</p>

<p align="center">
  <img src="./imgs/11.png" alt="imagen11" />
</p>

<p align="center">
  <img src="./imgs/12.png" alt="imagen12" />
</p>

# Desarrollo de la Solución Propuesta

In [1]:
# Se importan a continuación las librerías requeridas para esta aplicación:

from io import StringIO, BytesIO
import pandas as pd
import requests
import json
import io


In [2]:
# Se definen a continuación las variables para correr la aplicación:
fecha_inicio="2024-07-09" # define la fecha inicial desde la cual se solicita información de archivos de XM
fecha_fin="2024-07-20"    # define la fecha final hasta la cual se solicita información de archivos de XM
file_id = '' # acceso para descarga de archivos de las empresas generadoras

# llaves para conexión por API a XM para cargar balance de energia
api_key_1=""
api_key_2=""

# Paso 1: Extracciones

### Precios Bolsa

In [3]:
# Se consumen los datos desde la página de XM para el archivo de precios de bolsa:

url = f"https://www.simem.co/backend-files/api/PublicData?startDate={fecha_inicio}&endDate={fecha_fin}&datasetId=96D56E"
response = requests.get(url)

# Verificar si la solicitud fue exitosa
if response.status_code == 200:
    content_type = response.headers['Content-Type']
    #print(f"El archivo descargado es de tipo: {content_type}")
else:
    print("Error al descargar el archivo:", response.status_code)


In [4]:
#Se crea la variable data para almacenar el archivo json descargado desde XM. Luego se toman los datos "recors" del campo
#results y se crea, finalmente, el dataframe que contiene la información requeridad de precios de bolsa.

data = response.json()
records = data.get('result', {}).get('records', [])
dfPreciosBolsa = pd.DataFrame(records)        

### Plan de Despacho (Compromisos)

In [5]:
#Se consumen los datos desde la página de XM para determinar los compromisos de despacho:

url = f"https://www.simem.co/backend-files/api/PublicData?startdate={fecha_inicio}&enddate={fecha_fin}&datasetId=ff027b"
response = requests.get(url)

# Verificar si la solicitud fue exitosa
if response.status_code == 200:
    content_type = response.headers['Content-Type']
    #print(f"El archivo descargado es de tipo: {content_type}")
else:
    print("Error al descargar el archivo:", response.status_code)
    

In [6]:
#Se crea la variable data para almacenar el archivo json descargado desde XM. Luego se toman los datos "records" del campo
#results y se crea, finalmente, el dataframe que contiene la información requeridad de compromisos de despacho.

data = response.json()
records = data.get('result', {}).get('records', [])
dfCompriso = pd.DataFrame(records)        

### Capacidad Generadores

In [7]:
#En este apartado se hace el simil de descarga de la capacidad de despacho desde el generador(es) y se crea el dataframe de capacidad

url = f"https://drive.google.com/uc?id={file_id}"

response = requests.get(url)

dfCapacidad = pd.read_excel(BytesIO(response.content), header=5)


# Paso 2: Transformaciones

En este apartado se realizará las acciones de limpieza, eliminación de datos nulos, eliminación de datos no relevantes y las adecuaciones necesarias para poder hacer operaciones conjuntas necesarias entre los diferentes datasets (dataframes)

Se inicia con la homologación de nombres de columnas para los dataframes:

dfPreciosBolda:
-fecha: Fecha

dfDespachoUnidades:
-FechaHora: Fecha
-Valor: Compromiso(Kwh)

dfArchivoCapacidad:
-CODIGO: CodigoPlanta

In [8]:
dfPreciosBolsa = dfPreciosBolsa.rename(columns={
    'fecha': 'Fecha'
})

dfCompriso = dfCompriso.rename(columns={
    'FechaHora': 'Fecha',
    'Valor': 'Compromiso (Kwh)'
})

dfCapacidad = dfCapacidad.rename(columns={
    'CODIGO': 'CodigoPlanta',
    'FECHA': 'Fecha',
    'CAPACIDAD (Kwh)': 'Capacidad (Kwh)'
})

In [9]:
#Se elimina columna en dfArchivo capacidad que no tiene utilidad:

dfCapacidad = dfCapacidad.drop(columns=['Unnamed: 0'])

In [10]:
# Se eliminan valores nulos:

dfPreciosBolsa.dropna();
dfCompriso.dropna();
dfCapacidad.dropna();

In [11]:
#Se homologan los fomatos de fecha:

dfPreciosBolsa["Fecha"] = pd.to_datetime(dfPreciosBolsa["Fecha"])
dfCompriso["Fecha"] = pd.to_datetime(dfCompriso["Fecha"])
dfCapacidad["Fecha"] = pd.to_datetime(dfCapacidad["Fecha"])

In [12]:
#Se seleccionan los generadores correspondientes  al comercializadora ACME SAS

codigos_filtrar = ['ZPA2', 'ZPA3', 'ZPA4', 'ZPA5', 'GVIO', 'QUI1', 'CHVR']

dfCompromisoFiltrado = dfCompriso[dfCompriso['CodigoPlanta'].isin(codigos_filtrar)]

dfCapacidadFiltrado = dfCapacidad[dfCapacidad['CodigoPlanta'].isin(codigos_filtrar)]

In [13]:
# Se continua con la consolidación, en los dataframes, por fecha y unidad generadora (CodigoPlanta) y se totalizan las capacidades y los compromisos:

# Se convierte la columna Fecha solo a Fecha sin hora para poder agrupar

dfCapacidadFiltrado.loc[:, 'Fecha'] = dfCapacidadFiltrado['Fecha'].dt.date # se usa .loc para evitar el warning
dfCompromisoFiltrado.loc[:, 'Fecha'] = dfCompromisoFiltrado['Fecha'].dt.date # se usa .loc para evitar el warning



dfCapacidadAgrupado = dfCapacidadFiltrado.groupby(['CodigoPlanta', 'Fecha'])['Capacidad (Kwh)'].sum().reset_index()


dfCompromisoAgrupado = dfCompromisoFiltrado.groupby(['CodigoPlanta', 'Fecha'])['Compromiso (Kwh)'].sum().reset_index()


In [14]:
#Se realiza a continuación el balance entre capacidad y compromiso:

dfBalance = pd.merge(dfCapacidadAgrupado, dfCompromisoAgrupado, on=['CodigoPlanta', 'Fecha'], how='left')

dfBalance['Compromiso (Kwh)'] = dfBalance['Compromiso (Kwh)'].dropna()


# Crear la nueva columna 'Consolidado Planta' restando las capacidades
dfBalance['Consolidado (Kwh)'] = dfBalance['Capacidad (Kwh)'] - dfBalance['Compromiso (Kwh)']

dfBalance["Fecha"] = pd.to_datetime(dfBalance["Fecha"])

In [15]:
#Ahora se hace el balance valorizando la consolidación:

dfPreciosFiltrado = dfPreciosBolsa[(dfPreciosBolsa['CodigoVariable'] == 'PPBOGReal') & (dfPreciosBolsa['Version'] == 'TXR')]

# Hacer el merge de dfBalance con dfPreciosFiltrado en la columna 'fecha'
dfBalanceConPrecios = pd.merge(dfBalance, dfPreciosFiltrado[['Fecha', 'Valor']], on='Fecha', how='left')

# Crear la nueva columna 'Compromisos_MCOP' multiplicando 'Consolidado Planta' por 'Valor'
dfBalanceConPrecios['Compromisos_MMCOP'] = dfBalanceConPrecios['Consolidado (Kwh)'] * dfBalanceConPrecios['Valor']/1000000



In [16]:
# Finalmente, se da la indicación explicita de compra o venta de energía

dfBalanceConPrecios['Operación'] = dfBalanceConPrecios['Compromisos_MMCOP'].apply(lambda x: 'Compra' if x < 0 else 'Vender')




In [17]:
dfBalanceConPrecios.head()

,CodigoPlanta,Fecha,Capacidad (Kwh),Compromiso (Kwh),Consolidado (Kwh),Valor,Compromisos_MMCOP,Operación
0,GVIO,2024-07-10,24230440,12724160.0,11506280.0,219.7562,2528.576369,Vender
1,QUI1,2024-07-10,6570384,2962970.0,3607414.0,219.7562,792.751592,Vender
2,ZPA2,2024-07-10,655706,454000.0,201706.0,219.7562,44.326144,Vender
3,ZPA3,2024-07-10,467348,748000.0,-280652.0,219.7562,-61.675017,Compra
4,ZPA4,2024-07-10,544688,748000.0,-203312.0,219.7562,-44.679073,Compra


In [18]:
dfBalanceConPrecios['Valor'] = dfBalanceConPrecios['Valor'].round(2)
dfBalanceConPrecios['Compromisos_MMCOP'] = dfBalanceConPrecios['Compromisos_MMCOP'].round(2)

Ahora, previo a la etapa de carga de la información, se crea el archivo csv que será transferido

In [19]:
# Este archivo no se descargará. se creará en memoria para la operación de carga
csv_buffer = io.StringIO()

dfBalanceConPrecios.to_csv(csv_buffer, index=False)

csv_data = csv_buffer.getvalue()

# Se visualiza a continuación el contenido
print(csv_data)

CodigoPlanta,Fecha,Capacidad (Kwh),Compromiso (Kwh),Consolidado (Kwh),Valor,Compromisos_MMCOP,Operación
GVIO,2024-07-10,24230440,12724160.0,11506280.0,219.76,2528.58,Vender
QUI1,2024-07-10,6570384,2962970.0,3607414.0,219.76,792.75,Vender
ZPA2,2024-07-10,655706,454000.0,201706.0,219.76,44.33,Vender
ZPA3,2024-07-10,467348,748000.0,-280652.0,219.76,-61.68,Compra
ZPA4,2024-07-10,544688,748000.0,-203312.0,219.76,-44.68,Compra
ZPA5,2024-07-10,385940,748000.0,-362060.0,219.76,-79.56,Compra



## Paso 3:  Carga (Transferencia a XM)

In [20]:
#Se emplean las instrucciones para hacer la carga de la información mediante API

file_upload_access_token = ""
file_upload_account_id = ""
params = {'key1': api_key_1, 'key2': api_key_2}
response=requests.get("https://fastupload.io/api/v2/authorize", params)
json_response = json.loads(response.text)
try:
  file_upload_access_token = json_response["data"]["access_token"]
  file_upload_account_id = json_response["data"]["account_id"]
except:
  print("Error autenticando y autorizando en el servicio remoto de carga de archivos.")

In [21]:
upload_folder_id = ""  # Si tienes un folder ID

# Crear el archivo en memoria a partir de 'csv_data'
csv_in_memory = io.BytesIO(csv_data.encode('utf-8'))  # Convertir csv_data a bytes

# Configurar los archivos a subir
files = {"upload_file": ("dfReporteCompraVentaEnergiaAcme.csv", csv_in_memory, 'text/csv')}  # Nombrar el archivo

# Datos para la solicitud
data = {
    "access_token": file_upload_access_token,
    "account_id": file_upload_account_id,
    "folder_id": upload_folder_id  # Agregar folder_id si es necesario
}

# Enviar la solicitud POST a la API
response = requests.post("https://fastupload.io/api/v2/file/upload", files=files, data=data)

json_response = response.json()

# Extraer solo _status y _datetime
status = json_response.get('_status')
datetime = json_response.get('_datetime')

# Mostrar el resultado
print(f"Status: {status}")
print(f"Datetime: {datetime}")


Status: success
Datetime: 2024-09-05 20:16:02
